In [31]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


In [32]:
import os
import numpy as np
import scipy 
import tensorflow as tf
import imageio
import gzip
from PIL import Image

In [33]:
tf.logging.set_verbosity(tf.logging.INFO)

In [34]:
def cnn_model_fn(features, labels, mode):

    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional layer 1
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        filters = 32,
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu
    )

    # Pooling 1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional layer 2 and pooling layer
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu
    )

    # Pooling 2 with flattening
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)
    pool2_flat=tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Dense layer with dropout 
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout=tf.layers.dropout(inputs=dense, rate=0.4, training=mode==tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dropout, units=10)

    # Generate predictions
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Caluclate loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure training op
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}

    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [35]:
# Estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

# Logging predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, 
    every_n_iter=50)

# Our application logic will be added here

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x181ce93cc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [38]:
# Extract labels from MNIST labels into vector
def extract_labels(filename, num_images):
  with gzip.open(filename) as bytestream:
    bytestream.read(8)
    buf = bytestream.read(1 * num_images)
    labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
  return labels

train_labels = extract_labels("MNIST-data/train-labels-idx1-ubyte.gz", 60000)
eval_labels = extract_labels("MNIST-data/t10k-labels-idx1-ubyte.gz", 10000)
print(np.shape(train_labels))
print(np.shape(eval_labels))

(60000,)
(10000,)


In [48]:
# mnist = tf.contrib.learn.datasets.load_dataset("mnist")

# train_data_int = mnist.train.images
# train_labels_int = np.asarray(mnist.train.labels, dtype=np.int32)
# eval_data_int = mnist.test.images
# eval_labels_int = np.asarray(mnist.test.labels, dtype=np.int32)

# print(np.shape(train_data_int))
# print(np.shape(train_labels_int))
# print(np.shape(eval_data_int))
# print(np.shape(eval_labels_int))

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
(55000, 784)
(55000,)
(10000, 784)
(10000,)


In [104]:
# def parse_img(filename, label):
#     img_str = tf.read_file(filename)
#     img_dec = tf.image.decode_image(img_str)
# #     img_res = tf.image.resize_images(img_dec, [28, 28])
#     return img_dec, label

In [105]:
# img = "MNIST-processed-training/rot90/rot901.png"
# print(img)

In [115]:
# # Load rotated images for training
# filenames = tf.constant(["MNIST-processed-training/rot90/rot90{0}.png".format(k) for k in range(1,60001)])

# labels = train_labels
# dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
# dataset = dataset.map(parse_img)
    
# print(np.shape(dataset))

()


In [116]:
# dataset.output_shapes

(TensorShape(None), TensorShape([]))

In [ ]:
# # Load rotated images for training
# images = ["MNIST-processed-training/rot90/rot90{0}.png".format(k) for k in range(1,60001)]

# train_data = [] 
# for image in images: 
#     img = Image.open(image)
#     arr = np.array(img)
#     train_data.append(arr)
# #     train_data.append(imageio.imread(image))
    
# # Flatten images    
# for i in range(len(train_data)):
#     train_data[i] = np.array(train_data[i].flatten())
    
# print(np.shape(train_data))

In [36]:
# ROTATE 90 train
train_rot90 = np.zeros((60000,28,28), dtype=np.float32)
images_rot90 = ["MNIST-processed-training/rot90/rot90{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_rot90)):
    img = np.array(Image.open(images_rot90[i]))
    train_rot90[i, :, :] = img
 
print(np.shape(train_rot90))

# ROTATE 90 test
eval_rot90 = np.zeros((10000,28,28), dtype=np.float32)
images2_rot90 = ["MNIST-processed-test/rot90/test-rot90{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_rot90)):
    img = np.array(Image.open(images2_rot90[i]))
    eval_rot90[i, :, :] = img
 
print(np.shape(eval_rot90))

(60000, 28, 28)
(10000, 28, 28)


In [37]:
# INV train
train_inv = np.zeros((60000,28,28), dtype=np.float32)
images_inv = ["MNIST-processed-training/Inv/inv{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_inv)):
    img = np.array(Image.open(images_inv[i]))
    train_inv[i, :, :] = img
 
print(np.shape(train_inv))

# INV test
eval_inv = np.zeros((10000,28,28), dtype=np.float32)
images2_inv = ["MNIST-processed-test/inv/test-inv{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_inv)):
    img = np.array(Image.open(images2_inv[i]))
    eval_inv[i, :, :] = img
 
print(np.shape(eval_inv))

(60000, 28, 28)
(10000, 28, 28)


In [ ]:
# fliplr train
train_fliplr = np.zeros((60000,28,28), dtype=np.float32)
images_fliplr = ["MNIST-processed-training/fliplr/fliplr{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_fliplr)):
    img = np.array(Image.open(images_fliplr[i]))
    train_fliplr[i, :, :] = img
 
print(np.shape(train_fliplr))

# fliplr test
eval_fliplr = np.zeros((10000,28,28), dtype=np.float32)
images2_fliplr = ["MNIST-processed-test/fliplr/test-fliplr{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_fliplr)):
    img = np.array(Image.open(images2_fliplr[i]))
    eval_fliplr[i, :, :] = img
 
print(np.shape(eval_fliplr))

In [ ]:
# cutud train
train_cutud = np.zeros((60000,28,28), dtype=np.float32)
images_cutud = ["MNIST-processed-training/cutud/cutUD{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_cutud)):
    img = np.array(Image.open(images_cutud[i]))
    train_cutud[i, :, :] = img
 
print(np.shape(train_cutud))

# cutud test
eval_cutud = np.zeros((10000,28,28), dtype=np.float32)
images2_cutud = ["MNIST-processed-test/cutud/test-cutud{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_cutud)):
    img = np.array(Image.open(images2_cutud[i]))
    eval_cutud[i, :, :] = img
 
print(np.shape(eval_cutud))

In [ ]:
# checkerboard train
train_checkerboard = np.zeros((60000,28,28), dtype=np.float32)
images_checkerboard = ["MNIST-processed-training/checkerboard/fullcheck{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_checkerboard)):
    img = np.array(Image.open(images_checkerboard[i]))
    train_checkerboard[i, :, :] = img
 
print(np.shape(train_checkerboard))

# checkerboard test
eval_checkerboard = np.zeros((10000,28,28), dtype=np.float32)
images2_checkerboard = ["MNIST-processed-test/checkerboard/test-checkerboard{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_checkerboard)):
    img = np.array(Image.open(images2_checkerboard[i]))
    eval_checkerboard[i, :, :] = img
 
print(np.shape(eval_checkerboard))

In [ ]:
# flipud train
train_flipud = np.zeros((60000,28,28), dtype=np.float32)
images_flipud = ["MNIST-processed-training/flipud/flipud{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_flipud)):
    img = np.array(Image.open(images_flipud[i]))
    train_flipud[i, :, :] = img
 
print(np.shape(train_data))

# flipud test
eval_flipud = np.zeros((10000,28,28), dtype=np.float32)
images2_flipud = ["MNIST-processed-test/flipud/test-flipud{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_flipud)):
    img = np.array(Image.open(images2_flipud[i]))
    eval_flipud[i, :, :] = img
 
print(np.shape(eval_flipud))

In [ ]:
# invbot train
train_invbot = np.zeros((60000,28,28), dtype=np.float32)
images_invbot = ["MNIST-processed-training/invbot/invbot{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_invbot)):
    img = np.array(Image.open(images_invbot[i]))
    train_invbot[i, :, :] = img
 
print(np.shape(train_invbot))

# invbot test
eval_invbot = np.zeros((10000,28,28), dtype=np.float32)
images2_invbot = ["MNIST-processed-test/invbot/test-invbot{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_invbot)):
    img = np.array(Image.open(images2_invbot[i]))
    eval_invbot[i, :, :] = img
 
print(np.shape(eval_invbot))

In [49]:
# # Tensorflow MNIST data
# def main(unused_argv):
#     # Training
#     train_input_fn = tf.estimator.inputs.numpy_input_fn(
#         x={"x":train_data_int},
#         y=train_labels_int,
#         batch_size=100,
#         num_epochs=None,
#         shuffle=True)
    
#     mnist_classifier.train(
#         input_fn=train_input_fn,
#         steps=200,
#         hooks=[logging_hook])

#     # Evaluation
#     eval_input_fn = tf.estimator.inputs.numpy_input_fn(
#         x={"x": eval_data_int},
#         y=eval_labels_int,
#         num_epochs=1,
#         shuffle=False)
    
#     eval_results=mnist_classifier.evaluate(input_fn=eval_input_fn)
#     print(eval_results)

In [41]:
def main(unused_argv):
    # Training on rot90
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x":train_rot90},
        y=train_labels,
        batch_size=1000,
        num_epochs=None,
        shuffle=True)
    
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=200,
        hooks=[logging_hook])

    # Evaluation on rot90
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_rot90},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    
    eval_results=mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [42]:
if __name__ == "__main__":
  tf.app.run()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-4201
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 4202 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.00000215 0.99941874 0.00001955 ... 0.00028886 0.00002232 0.00000258]
 [0.         0.00000005 0.0000252  ... 0.00000028 0.0000002  0.00000024]
 [0.8263984  0.00000012 0.16598742 ... 0.00005144 0.00257748 0.00018237]
 ...
 [0.00000948 0.00000874 0.00018036 ... 0.00018195 0.99227834 0.00678182]
 [0.00000207 0.00026185 0.06254628 ... 0.00136261 0.00472883 0.05135954]
 [0.00000341 0.00000006 0.00000002 ... 0.00000001 0.00000158 0.0000002 ]]
INFO:tensorflow:loss = 0.20579624, step = 4202
INFO:tensorflow:probabilities = [[0.00000258 0.         0.00046831 ... 

SystemExit: 

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#model.layers[-1].get_weights()

In [27]:
mnist_classifier.get_variable_value( 'logits')



NotFoundError: Key logits not found in checkpoint

In [30]:
 tf.global_variables()

[]